In [ ]:
import torch
torch.cuda.is_available() # 実行環境でcuda（GPUを利用するためのツール）が使えるかを表示


True

In [ ]:
'''前処理用のモジュールをインポートする'''
import os
import glob
import codecs
import logging
import re
import random
from itertools import chain

'''形態素解析用の外部モジュールをインポートする'''
!pip install janome
from janome.tokenizer import Tokenizer

'''Word2Vec用のモジュールをインポートする'''
from gensim.models import Word2Vec

'''LSTM用のモジュールをインポートする'''
import copy
import numpy as np
from numpy.random import rand # ランダムなベクトルを作るための関数
from tqdm import tqdm # ループの進捗を表示するモジュール
import torch # 深層学習用のモジュールPyTorch
import torch.nn as nn #　ニューラルネットワーク用のモジュール
import torch.nn.functional as F # 活性化関数用のモジュール
import torch.optim as optim # PyTorchのオプティマイザー
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


     |████████████████████████████████| 19.7MB 1.3MB/s 


In [ ]:
try:
  from google.colab import drive # Google Driveからファイルを読み込むための外部モジュール
  drive.mount('./gdrive')
  # base_dir上に各作者のディレクトリ['natsume', 'dazai', 'akutagawa', 'fukuzawa', 'mori']があることを想定
  base_dir = './gdrive/My Drive/DSAP/DSAP_Presentation/dataset'
# google.colabモジュールが存在しない場合（ローカル環境など）であればbase_dirをスクリプトと同じディレクトリに指定
except ModuleNotFoundError: # Google Colab上で実行しない場合
  base_dir = './'


Mounted at ./gdrive


In [ ]:
RUBY_PATTERN = re.compile(u'《[^》]+》') # ルビの正規表現パターン
ADDITION_PATTERN = re.compile(' ［.*\］| （.*\）') # 文章中の補足情報のパターン（カッコは全角）
ALPHABET_PATTERN = re.compile(u'[a-zA-Z]') # アルファベットの正規表現パターン

def delete_newline_char(line):
  '''改行コードをすべて削除する'''
  return line.replace('\r', '').replace('\n', '')

def remove_ruby(line):
  '''ルビを削除する'''
  return RUBY_PATTERN.sub('', line)

def remove_additions(line):
  '''補足情報を消す'''
  return ADDITION_PATTERN.sub('', line)

def remove_escape_char(line):
  '''エスケープされた文字を消す'''
  return line.replace('\u3000', '')

def remove_symbols(line):
  '''記号を消す'''
  line = line.replace('-', '').replace(':', '').replace('/', '').replace('(', '').replace(')', '')
  return line

def remove_alphabets(line):
  '''アルファベットを消す'''
  return ALPHABET_PATTERN.sub('', line)

def separate_line_with_puncs(lines):
  '''句点で区切った文を吐き出す'''
  for line in lines:
    for sentence in line.split('。'):
      yield sentence

def preprocess(data):
  '''文章を前処理して文のリストにする'''
  lines = map(delete_newline_char, data)
  lines = map(remove_ruby, lines)
  lines = map(remove_additions, lines)
  lines = map(remove_escape_char, lines)
  lines = map(remove_symbols, lines)
  sentences = separate_line_with_puncs(lines)
  '''文が空のものを除去する'''
  sentences = filter(lambda x: len(x), sentences)
  return list(sentences)
  

In [ ]:
WRITERS = ['natsume', 'dazai', 'akutagawa', 'fukuzawa', 'mori'] # 作者のリスト
train_sentences = dict() # 訓練データの辞書（キーは作者の名前、要素は文のリスト）
test_sentences = dict() # テストデータの辞書（キーは作者の名前、要素は文のリスト）
for writer in WRITERS: # ['natsume', 'dazai', 'akutagawa', fukuzawa', 'mori']
  if not os.path.isdir(os.path.join(base_dir, writer)): # 作者名のディレクトリがあるかどうかをチェック
    continue
  train_sentences[writer] = list() # 訓練用の各作者の文を入れるリスト
  test_sentences[writer] = list() # テスト用の各作者の文を入れるリスト
  '''各ディレクトリの全テキストファイルを取得'''
  files = glob.glob(os.path.join(base_dir, writer, '*.txt'))
  '''上で得たテキストファイルを4:1で訓練データとテストデータにする'''
  train_files = files[:len(files) * 4 // 5]
  test_files = files[len(files) * 4 // 5:]

  '''ファイルから文章を読み込み、上で定義したpreprocess関数に通してリストに格納していく'''
  for train_file in train_files:
    with codecs.open(train_file, 'r', 'shift_jis') as f_r:
      train_sentences[writer].extend(preprocess(f_r.readlines()))
  for test_file in test_files:
    with codecs.open(test_file, 'r', 'shift_jis') as f_r:
      test_sentences[writer].extend(preprocess(f_r.readlines()))


In [ ]:
'''各作家の文リストのうち、要素数が一番少ないものの要素数を取得する'''
min_train = min(len(sentences) for sentences in train_sentences.values())
for writer in train_sentences.keys():
  '''上で得た要素数に応じて、他の作者の文を減らす。この時、訓練データをシャッフルする'''
  random.shuffle(train_sentences[writer])
  train_sentences[writer] = train_sentences[writer][:min_train]


In [ ]:
tokenizer = Tokenizer() # 形態素解析用のインスタンスを作成

separated_train_sentences = dict() # 形態素解析した訓練データを格納する辞書
separated_test_sentences = dict() # 形態素解析したテストデータを格納する辞書

for writer in WRITERS:
  separated_train_sentences[writer] = list() # 訓練用の各作者の文を形態素解析したものを入れるリスト
  separated_test_sentences[writer] = list() # テスト用の各作者の文を形態素解析したものを入れるリスト

'''訓練データについて各作者のすべての文について形態素解析を行い、新たに文ごとに形態素のリストを作成する'''
for writer, sentences in train_sentences.items():
  separated_train_sentences[writer] = [[token.surface for token in tokenizer.tokenize(sentence)] for sentence in sentences]

'''テストデータについて各作者のすべての文について形態素解析を行い、新たに文ごとに形態素のリストを作成する'''
for writer, sentences in test_sentences.items():
  separated_test_sentences[writer] = [[token.surface for token in tokenizer.tokenize(sentence)] for sentence in sentences]


In [ ]:
'''Word2Vecのログを表示する際のフォーマットを指定します'''
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
'''訓練データについてWord2Vecのモデルを作成し学習させます'''
word_model = Word2Vec(chain.from_iterable(separated_train_sentences.values()), size=52, min_count=1, window=5, iter=20)


2021-01-08 02:08:57,435 : INFO : collecting all words and their counts
2021-01-08 02:08:57,436 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-01-08 02:08:57,473 : INFO : collected 15455 word types from a corpus of 179174 raw words and 6735 sentences
2021-01-08 02:08:57,474 : INFO : Loading a fresh vocabulary
2021-01-08 02:08:57,503 : INFO : effective_min_count=1 retains 15455 unique words (100% of original 15455, drops 0)
2021-01-08 02:08:57,506 : INFO : effective_min_count=1 leaves 179174 word corpus (100% of original 179174, drops 0)
2021-01-08 02:08:57,546 : INFO : deleting the raw counts dictionary of 15455 items
2021-01-08 02:08:57,548 : INFO : sample=0.001 downsamples 41 most-common words
2021-01-08 02:08:57,548 : INFO : downsampling leaves estimated 119946 word corpus (66.9% of prior 179174)
2021-01-08 02:08:57,585 : INFO : estimated required memory for 15455 words and 52 dimensions: 14156780 bytes
2021-01-08 02:08:57,586 : INFO : resetting layer

In [ ]:
word_model_path = os.path.join(base_dir, 'word2vec.gensim.model') # 学習済みモデルのパス
'''学習済みモデルをロードする'''
word_model = Word2Vec.load(word_model_path)
'''上記でロードしたモデルについて、自分で用意した訓練データを入れて再学習させる'''
word_model.train(chain.from_iterable(separated_train_sentences.values()), total_examples=sum([len(sentences) for sentences in separated_train_sentences.values()]), epochs=100)


2021-01-08 02:19:51,066 : INFO : loading Word2Vec object from ./gdrive/My Drive/DSAP/DSAP_Presentation/dataset/word2vec.gensim.model
2021-01-08 02:19:52,057 : INFO : loading wv recursively from ./gdrive/My Drive/DSAP/DSAP_Presentation/dataset/word2vec.gensim.model.wv.* with mmap=None
2021-01-08 02:19:52,058 : INFO : loading syn0 from ./gdrive/My Drive/DSAP/DSAP_Presentation/dataset/word2vec.gensim.model.wv.syn0.npy with mmap=None
2021-01-08 02:19:54,559 : INFO : loading syn1neg from ./gdrive/My Drive/DSAP/DSAP_Presentation/dataset/word2vec.gensim.model.syn1neg.npy with mmap=None
2021-01-08 02:19:57,228 : INFO : Model saved using code from earlier Gensim Version. Re-loading old model in a compatible way.
2021-01-08 02:19:57,230 : INFO : loading Word2Vec object from ./gdrive/My Drive/DSAP/DSAP_Presentation/dataset/word2vec.gensim.model
2021-01-08 02:19:58,134 : INFO : loading wv recursively from ./gdrive/My Drive/DSAP/DSAP_Presentation/dataset/word2vec.gensim.model.wv.* with mmap=None
20

(125167, 179174)

In [ ]:
print(word_model.wv['人間'])


[ 0.02454424  0.24313766 -0.11602586  0.1068626  -0.00916309  0.12964547
 -0.57041395  0.37440494 -0.46154094 -0.41853964  0.3872797  -0.03984099
 -0.46275315  0.25286174 -0.20168215  0.3887556   0.21822833 -0.08316327
 -0.33720157  0.01387228  0.79599243  0.39081353 -0.40288326  0.23015828
 -0.13524047 -0.10694273 -0.0375444  -0.00601401  0.45110646  0.03807493
  0.71882373  0.10643985 -0.3844638   0.5986704  -0.61028177  0.1677802
  0.2496373   0.6166424   0.55098814 -0.553066   -0.2972701   0.09899536
  0.36897358  0.23357865  0.9184577  -0.39860702 -0.2824983   1.0004365
 -0.08955841  0.35711867]


In [ ]:
print(word_model.wv.most_similar(['人間']))


2021-01-08 02:20:21,278 : INFO : precomputing L2-norms of word weight vectors


[('本性', 0.8012143969535828), ('自我', 0.8007199764251709), ('もの', 0.7981474995613098), ('他人', 0.7979690432548523), ('肉体', 0.7919148802757263), ('無意識', 0.7917479872703552), ('他者', 0.7867447733879089), ('世の中', 0.7861497402191162), ('心', 0.7839295864105225), ('宿命', 0.7813477516174316)]


In [ ]:
torch.manual_seed(1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epoch_num = 100 # 学習を繰り返す回数
batch_size = 100 # バッチサイズ、一度にどれだけの数のデータを一緒に扱って学習するかを決めるパラメータ
lr = 0.001 # 学習率、パラメータを更新する量
embedding_dim = word_model.trainables.layer1_size # 単語の分散表現の次元


In [ ]:
class LSTM(nn.Module):
  '''LSTMを定義するクラス'''

  def __init__(self, embedding_dim, hidden_dim=50): # embedding_dimは入力の次元数

    super(LSTM, self).__init__()
    self.hidden_dim = hidden_dim # 隠れ層の次元
    self.tagset_size = len(WRITERS) # 出力の次元数（作者5人であれば5）

    '''LSTMのモデルを定義する。隠れ層は一層で、biLSTMを使用する'''
    self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                        num_layers=1, bidirectional=True)

    '''全結合層を定義する'''
    self.fc = nn.Linear(hidden_dim * 2, self.tagset_size)

    '''隠れ層を定義する'''
    self.hidden = self._init_hidden()

  def _init_hidden(self): # 隠れ層の初期化を行う関数
    return (torch.randn(2, 1, self.hidden_dim // 2).to(device),
            torch.randn(2, 1, self.hidden_dim // 2).to(device))

  def forward(self, sentence):
    '''入力をLSTMに通す'''
    lstm_out, _ = self.lstm(sentence, self.hidden)
    '''LSTMからの出力を平均プーリングと最大値プーリングに通して結合する'''
    cat_out = torch.cat((torch.max(lstm_out, 0).values, torch.mean(lstm_out, 0)), dim=1)
    '''全結合させる'''
    fc_out = self.fc(cat_out)
    output = F.log_softmax(fc_out, dim=1)
    return output


In [ ]:
def result_log(save_results=False):

  '''正解ラベルの一覧を取得する'''
  y_true = [WRITERS[data[1]] for data in test_data]
  '''推測したラベルの一覧を格納するリストを定義する'''
  y_pred = list()
  '''テストデータで推論を行う際はパラメータの更新を行わないようにする'''
  with torch.no_grad():
    correct_answers = 0
    for sentence, writer_index in test_data:

      '''テストデータの中身をベクトルで置き換える'''
      '''Word2Vecのモデルに存在しない単語が出てきた場合は適当なベクトルで置き換える'''
      sentence_vec = [word_model.wv[word] if word in word_model.wv.vocab
                     else rand(embedding_dim).astype(np.float32) for word in sentence]

      '''すぐ上で用意したベクトルのPyTorchのテンソルに変換する'''
      sentence_in = torch.tensor(sentence_vec).view(len(sentence), 1, -1).to(device)

      '''テストデータについてLSTMに通して推論を行う'''
      output = lstm_model(sentence_in)
      _, pred = torch.max(output, 1)
      y_pred.append(pred)
      target = torch.tensor([writer_index]).to(device)
      if pred == target:
        correct_answers += 1

    print('Test data accuracy: ', correct_answers / len(test_data))
    '''推論結果をNumPyの配列に変換する'''
    y_pred = torch.tensor(y_pred).numpy()
    '''推論結果から作者名を取り出す'''
    y_pred = [WRITERS[pre] for pre in y_pred]
    '''各作者についての詳しいレポートを出力する'''
    report = classification_report(y_true, y_pred)
    '''推論結果と実際のデータに関する混同行列を出力する'''
    cof_mat = confusion_matrix(y_true, y_pred)
    print(report)
    print(cof_mat)
    if save_results:
      with open(os.path.join(base_dir, 'classification_report.txt'), 'w') as f_w:
        f_w.write(report)
      with open(os.path.join(base_dir, 'cof_mat.txt'), 'w') as f_w:
        f_w.write(str(cof_mat))

def save_model(model, epoch=None):
# モデルを保存する関数。10エポックごとと学習終了後に保存
  if epoch:
    path = os.path.join(base_dir, str(epoch) + '-model.pth')
  else:
    path = os.path.join(base_dir, 'model.pth')
  torch.save(model.state_dict(), path)


In [ ]:
train_data = list() # LSTMに入れる訓練データのリスト
test_data = list() # LSTMに入れるテストデータのリスト

'''入力するデータを形態素解析した文とタグのタプルをリストにする'''
for writer, sentences in separated_train_sentences.items():
  writer_index = WRITERS.index(writer)
  for sentence in sentences:
    train_data.append([sentence, writer_index])

'''入力するデータを形態素解析した文とタグのタプルをリストにする'''
for writer, sentences in separated_test_sentences.items():
  writer_index = WRITERS.index(writer)
  for sentence in sentences:
    test_data.append([sentence, writer_index])


In [ ]:
'''LSTMモデルを生成する'''
lstm_model = LSTM(embedding_dim).to(device)

'''optimizerを定義する'''
optimizer = optim.Adam(lstm_model.parameters(), lr=lr)
'''訓練データの個数から1エポック中のループ回数を決定する'''
loop_num = len(train_data) // batch_size


In [ ]:
for epoch in range(epoch_num):
  all_loss = 0
  random.shuffle(train_data) # 毎エポック訓練データの中身をシャッフル
  correct_answers = 0
  print('epoch: ', epoch)
  for i in tqdm(range(0, loop_num)):
    index = i * batch_size
    batch_data = [train_data[j] for j in range(index, index+batch_size)]

    for sentence, writer_index in batch_data:
      lstm_model.zero_grad()

      '''訓練データの中身をベクトルで置き換える'''
      '''Word2Vecモデルに存在しない単語が出てきた場合は適当なベクトルで置き換える'''
      sentence_vec = [word_model.wv[word] if word in word_model.wv.vocab
                     else rand(embedding_dim).astype(np.float32) for word in sentence]

      '''すぐ上で用意したベクトルをPyTorchのテンソルに変換する'''
      sentence_in = torch.tensor(sentence_vec).view(len(sentence), 1, -1).to(device)

      '''同様にラベルの方もPyTorchのテンソルに変換する'''
      target = torch.tensor([writer_index]).to(device)
      output = lstm_model(sentence_in)

      _, pred = torch.max(output, 1)
      if pred == target:
        correct_answers += 1

      loss = F.nll_loss(output, target) # lossを計算
      loss.backward() # 誤差逆伝搬の計算
      optimizer.step() # lossに応じてパラメータを更新
      all_loss += loss.item()

  print('Training data accuracy: ', correct_answers / (loop_num * batch_size))
  print('Loss: ', str(all_loss))

  if epoch % 10 == 0:
    save_model(lstm_model, epoch)
    result_log(save_results=True)
  else:
    result_log()

result_log(save_results=True)
save_model(lstm_model)


  0%|          | 0/67 [00:00<?, ?it/s]

epoch:  0


100%|██████████| 67/67 [00:30<00:00,  2.18it/s]


Training data accuracy:  0.5783582089552238
Loss:  6675.711295971414
Test data accuracy:  0.4668850072780204


  0%|          | 0/67 [00:00<?, ?it/s]

...

Training data accuracy:  0.9876119402985075
Loss:  254.41562058541092
Test data accuracy:  0.5661693352741388


  0%|          | 0/67 [00:00<?, ?it/s]

              precision    recall  f1-score   support

   akutagawa       0.04      0.18      0.06       261
       dazai       0.52      0.62      0.56      2699
    fukuzawa       0.75      0.79      0.77       634
        mori       0.43      0.44      0.43      3764
     natsume       0.74      0.60      0.66      9130

    accuracy                           0.57     16488
   macro avg       0.49      0.53      0.50     16488
weighted avg       0.62      0.57      0.59     16488

[[  46  112    1   40   62]
 [ 231 1669   22  228  549]
 [   6   24  504   65   35]
 [ 332  447   16 1643 1326]
 [ 663  983  128 1883 5473]]
epoch:  99


100%|██████████| 67/67 [00:31<00:00,  2.13it/s]


Training data accuracy:  0.9888059701492538
Loss:  219.93552238559906
Test data accuracy:  0.5188622028141678
              precision    recall  f1-score   support

   akutagawa       0.03      0.20      0.06       261
       dazai       0.48      0.63      0.55      2699
    fukuzawa       0.77      0.65      0.70       634
        mori       0.39      0.48      0.43      3764
     natsume       0.75      0.50      0.60      9130

    accuracy                           0.52     16488
   macro avg       0.48      0.49      0.47     16488
weighted avg       0.61      0.52      0.55     16488

[[  53  114    0   41   53]
 [ 309 1689   15  272  414]
 [   4   67  412  124   27]
 [ 438  453   17 1808 1048]
 [ 836 1167   92 2442 4593]]
Test data accuracy:  0.5168607472100922
              precision    recall  f1-score   support

   akutagawa       0.03      0.20      0.05       261
       dazai       0.48      0.63      0.54      2699
    fukuzawa       0.77      0.65      0.70       634
   